In [6]:
pip install ligthgbm
from datetime import date
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import random 
import warnings
import operator
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
import math

SyntaxError: invalid syntax (<ipython-input-6-766094a5d2f7>, line 1)

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

###### def load_data():
    loc_train = "../input/forest-cover-type-kernels-only/train.csv"
    loc_test = "../input/forest-cover-type-kernels-only/test.csv"
    loc_submission = "kaggle.rf200.entropy.submission.csv"
    df_train = pd.read_csv("../input/train.csv")
    df_test = pd.read_csv("../input/test.csv")
    return (loc_train, loc_test, loc_submission, df_train,df_test)

loc_train, loc_test, loc_submission, df_train,df_test = load_data()

In [ ]:
cols_to_normalize = ['Aspect','Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology','Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points']
df_train[cols_to_normalize] = normalize(df_train[cols_to_normalize])
df_test[cols_to_normalize] = normalize(df_test[cols_to_normalize])

In [ ]:
feature_cols = [col for col in df_train.columns if col not in ['Cover_Type','Id']]
feature_cols.append('binned_elevation')
feature_cols.append('Horizontal_Distance_To_Roadways_Log')
feature_cols.append('Soil_Type12_32')
feature_cols.append('Soil_Type23_22_32_33')
feature_cols.append('Horizontal_Distance_To_Hydrology')

In [ ]:
df_train['binned_elevation'] = [math.floor(v/50.0) for v in df_train['Elevation']]
df_test['binned_elevation'] = [math.floor(v/50.0) for v in df_test['Elevation']]

df_train['Horizontal_Distance_To_Roadways_Log'] = [math.log(v+1) for v in df_train['Horizontal_Distance_To_Roadways']]
df_test['Horizontal_Distance_To_Roadways_Log'] = [math.log(v+1) for v in df_test['Horizontal_Distance_To_Roadways']]

df_train['Soil_Type12_32'] = df_train['Soil_Type32'] + df_train['Soil_Type12']
df_test['Soil_Type12_32'] = df_test['Soil_Type32'] + df_test['Soil_Type12']
df_train['Soil_Type23_22_32_33'] = df_train['Soil_Type23'] + df_train['Soil_Type22'] + df_train['Soil_Type32'] + df_train['Soil_Type33']
df_test['Soil_Type23_22_32_33'] = df_test['Soil_Type23'] + df_test['Soil_Type22'] + df_test['Soil_Type32'] + df_test['Soil_Type33']

df_train['Horizontal_Distance_To_Hydrology_Log'] = [math.log(v+1) for v in df_train['Horizontal_Distance_To_Hydrology']]
df_test['Horizontal_Distance_To_Hydrology_Log'] = [math.log(v+1) for v in df_test['Horizontal_Distance_To_Hydrology']]
X_train = df_train[feature_cols]
X_test = df_test[feature_cols]
y= df_train['Cover_Type']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.10, random_state=42, stratify=y)
X_train.shape,y_train.shape,X_val.shape

In [ ]:
%%time
from bayes_opt import BayesianOptimization
import lightgbm as lgb
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_roun=25, n_folds=7, random_seed=42, n_estimators=10000, learning_rate=0.02, output_process=False,colsample_bytree=0.93,min_child_samples=56,subsample=0.84):
    #prepare data
    train_data = lgb.Dataset(data=X, label=y)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight, colsample_bytree,min_child_samples,subsample):
        params = {'application':'multiclass','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':300, 'metric':'macroF1'}
        params["num_leaves"] = int(round(num_leaves))
        params["num_class"] = 8
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        params['colsample_bytree'] = 0.93
        params['min_child_samples'] = 56,
        params['subsample'] = 0.84
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (19, 45),'feature_fraction': (0.1, 0.9),'bagging_fraction': (0.8, 1),'max_depth': (5, 8.99),'lambda_l1': (0, 5),'lambda_l2': (0, 3),'min_split_gain': (0.001, 0.1),'min_child_weight': (5, 50),'colsample_bytree' : (0.7,1.0),'min_child_samples' : (40,65), 'subsample' : (0.7,1.0) }, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_roun)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    # return best parameters
    return lgbBO.res['max']['max_params']

opt_params = bayes_parameter_opt_lgb(X_train, y_train, init_round=10, opt_roun=10, n_folds=6, random_seed=42, n_estimators=500, learning_rate=0.02,colsample_bytree=0.93)

In [ ]:
opt_params = {'bagging_fraction': 0.9957236684465528,
 'colsample_bytree': 0.7953949538181928,
 'feature_fraction': 0.7333800304661316,
 'lambda_l1': 1.79753950286893,
 'lambda_l2': 1.710590311253639,
 'max_depth': 6,
 'min_child_samples': 48,
 'min_child_weight': 49,
 'min_split_gain': 0.016737988780906453,
 'num_leaves': 33,
 'subsample': 0.9033449610388691}

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb



In [ ]:
lgb = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.1, objective='multiclass',random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000, class_weight='balanced')
lgb.set_params(**opt_params)
lgb.fit(X_train,y_train)
print(lgb.score(X_train,y_train))
y_pred = lgb.predict(X_test)

In [ ]:
#sub = pd.read_csv("../input/forest-cover-type-kernels-only/sample_submission.csv")
#sub.head()
#submission = pd.DataFrame({'Id':X_test.Id,'Cover_Type':y_pred})
#submission.to_csv('SVM_submission.csv',index=False)
#X_test.head()
sub = {'Id':df_test["Id"].values,'Cover_Type':y_pred.astype('int')}
sub = pd.DataFrame(data=sub)
cols = ['Id',"Cover_Type"]
sub = sub[cols]
sub


In [ ]:
sub["Cover_Type"] = y_pred
sub.to_csv("submission_works.csv", index = False)